# Analysis with the rest feature

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
df=pd.read_csv("../data/games_mongodump.csv")
df.head(5)

,date_time,home_team,away_team,odd_home,odd_away,score_home,score_away,overtime
0,2015-03-13T01:00:00.000Z,Washington Wizards,Memphis Grizzlies,1.259740,4.030000,107,87,False
1,2015-03-12T04:30:00.000Z,Golden State Warriors,Detroit Pistons,1.050000,11.520000,105,98,False
2,2015-03-12T04:30:00.000Z,Portland Trail Blazers,Houston Rockets,1.729927,2.160000,105,100,False
3,2015-03-12T04:00:00.000Z,Phoenix Suns,Minnesota Timberwolves,1.490196,2.720000,106,97,False
4,2015-03-12T03:00:00.000Z,Denver Nuggets,Atlanta Hawks,3.450000,1.330033,115,102,False
